
Editor | Date 
--- | --- 
Alireza Ranjbar | 27.03.2019 

# Main functions:

- __Auto_mask_image__
    given an image path, returns a Dataframe with information such as coordinates of the masks,and if saving path is defined for it, it will save the DataFrame as an xlsx file.

    - __inner_table_texts__
        - First it uses __scikitting(black_region=0, unite_boxes=0)__ to find the table fields.(It finds closed white regions) 
            -  For each region from above(each table field) it applies __scikitting(black_region=1, unite_boxes=1, minDist=40)__ to find closed black regions(text characters), and uses __unite_close_boxes__ to merge them together if they are close by __minDist__ distance or overlap.
            - It then returns a DataFrame containing the coordinates of the merged text boxes.
        - Now the texts outside of the tables should be found. For that we first set all pixels of the tables found in the first step to white and output it as __outer_img_array__.
    - __outer_table_texts__ gets __outer_img_array__ after being cleaned/eroded, and uses  __scikitting(black_region=1, unite_boxes=1, minDist=40)__ to find text characters of outside of the tables. 
    - The DataFrame containing the text boxes out of the table is then concatanated with the ones from inside of the tables.
    - __clean_and_calculate_columns__ gets the final DataFrame of text coordinates and remove boxes whose geometries(area and aspect ratio) does not make sense. It also writes the information of such geometries in the DataFrame.
    
- __show_labels__ takes the output DataFrame of __Auto_mask_image__ along with the image array, and plots the masks

<img src="images/Automask_flow.png">
<img src="images/work flow.png">

#### Coordinates legend:
- image_array[top, left] returns the array element on top-left of the bounding bod
- image_array[Y_coordinate2, X_coordinate2] returns the array element on bottom-right of the bounding bod
- __img_mask_cut(image_array, box)__  returns the array_cut of the mask given a row of the DataFrame which has the coordinates:

    image_array[ 
               box['top']: box['Y_coordinate2'],   
               box['left']: box['X_coordinate2']
               ]

In [2]:
# The complete script:
%run img_auto_mask.py

In [3]:
# image example used for this document:
IMG_path=r'C:.jpg'
image_array = cv2.imread(IMG_path, 0)

In [7]:
all_boxes = Auto_mask_image(img_path=IMG_path, saving_path=None)
all_boxes

,top,left,Y_coordinate2,X_coordinate2,row_id,group_id,column_id,upper_label,left_label,Aspect_ratio(width/height),Area,width,height,center_x,center_y,text
0,120,2048,199,2206,1.0,0,None,None,None,2.000000,6241.0,158.0,79.0,2127.0,159.5,None
1,130,996,190,1668,1.0,1,None,None,None,11.200000,3600.0,672.0,60.0,1332.0,160.0,None
2,130,259,190,779,1.0,2,None,None,None,8.666667,3600.0,520.0,60.0,519.0,160.0,None
3,202,2048,387,2204,1.0,3,None,None,None,0.843243,34225.0,156.0,185.0,2126.0,294.5,None
4,197,259,245,788,1.0,4,None,None,None,11.020833,2304.0,529.0,48.0,523.5,221.0,None
5,197,994,245,1576,1.0,5,None,None,None,12.125000,2304.0,582.0,48.0,1285.0,221.0,None
6,339,3558,384,3960,1.0,6,None,None,None,8.933333,2025.0,402.0,45.0,3759.0,361.5,None
7,330,258,392,733,1.0,7,None,None,None,7.661290,3844.0,475.0,62.0,495.5,361.0,None
8,330,996,392,1580,1.0,8,None,None,None,9.419355,3844.0,584.0,62.0,1288.0,361.0,None
9,335,3068,384,3462,1.0,9,None,None,None,8.040816,2401.0,394.0,49.0,3265.0,359.5,None


In [1]:
#show_labels(all_boxes, image_array, show=1, saving_name=None, save_loc=None)

- __read_write_txts__
    - It gets the DataFarame of box coordinates from __Auto_mask_image__ and also gets the array of the original image then uses __pytesseract__ to read the texts inside the boxes and writes them in the DataFrame.

In [3]:
#all_boxes_with_text = read_write_txts(cv2.imread(loc + name, 0), all_boxes)
#all_boxes_with_text

- __get_upper_labels__ 
    - __until_n_boxes__ above the __target_box__ (pandas row serries from the above DataFrame) that have have at least __n_letters_thr__ letters inside them, it it returns a sorted list of these texts.
- __get_left_labels__
    - __until_n_boxes__ left side of the __target_box__ (pandas row serries from the above DataFrame) that have have at least __n_letters_thr__ letters inside them, it returns a sorted list of these texts.
    
   
 - example: assume our __target_box__ has the coordinates of top == 2456 and left == 1426

In [4]:
#target_box = all_boxes.query('top == ' +str(2456)+' & ' + 'left == '+str(1426)).iloc[0]
#target_box

In [5]:
#show_labels(target_box.to_frame().T, image_array, show=1, saving_name=None, save_loc=None)

In [6]:
#print('target field text: ', target_box.text)
#print('Upper labels: ', get_upper_labels(all_boxes, target_box, until_n_boxes=3, n_letters_thr=3))
#print('Left labels: ', get_left_labels(all_boxes, target_box, until_n_boxes=3, n_letters_thr=3))

# Sub functions 


- __errosion_and_extrapolation__ takes an image array and tries to remove horizontal and vertical lines.

In [7]:
#errosion_and_extrapolation(image_array = image_array)

- __scikitting__ uses scikit-image to find regions, and optionally merges the found regions
    - if __make_img_label_overlay__, it returns an array of the image, colored by the detected regions. This is a time consuming part and good only for plotting.
    - if __black_region__, it looks if it can draw borders in whight regions around a black regions. Otherwise (for finding table fileds) it tries to draw a border in black regions around white regions
    - if __unite_boxes__, it merges the found masks using if the are close by __minDist__ distance or overlap. For meging __unite_close_boxes__ function is used inside __scikitting__ function
    - __area_threshold__ is the minimum area for the found regions inside the image to be accepted and saved in the DataFrame.
    - __thresholding__ is the thresholding method used for the initial image processing. Options: __'mean'__(default), __'niblack'__

In [8]:
"""df, image_label_overlay=scikitting(image_array,
                                    black_region=False,
                                    clearborder=1,
                                    unite_boxes=0,
                                    minDist=40,
                                    thresholding='mean'
                                    area_threshold=5000,
                                    make_img_label_overlay=1
                                    )"""
#show_labels(df, image_label_overlay, show=1, saving_name=None, save_loc=None)

"df, image_label_overlay=scikitting(image_array,\n                                    black_region=False,\n                                    clearborder=1,\n                                    unite_boxes=0,\n                                    minDist=40,\n                                    thresholding='mean'\n                                    area_threshold=5000,\n                                    make_img_label_overlay=1\n                                    )"

In [71]:
df

,top,left,Y_coordinate2,X_coordinate2,row_id,group_id,column_id,upper_label,left_label
0,2161,274,2408,4012,2,None,None,None,None
1,2414,272,3205,4012,3,None,None,None,None
2,3214,3047,3294,3441,4,None,None,None,None
3,3214,3604,3295,4009,4,None,None,None,None
4,3410,3334,3518,3992,5,None,None,None,None
5,3527,3333,4069,3992,6,None,None,None,None
6,4079,3331,5018,3992,7,None,None,None,None
7,4637,266,4738,3265,7,None,None,None,None
8,4747,283,4802,3265,7,None,None,None,None
9,4810,264,4865,3265,7,None,None,None,None


In [9]:
#if black_region== True 
#and area area_threshold is changed from 5000 to 400
#and unite_boxes==0 (we dont merge the boxes otherwise it would be similar to using outer_table_texts function.

"""df, image_label_overlay=scikitting(image_array,
                                    black_region=True,
                                    clearborder=1,
                                    unite_boxes=0,
                                    minDist=40,
                                    area_threshold=200,
                                    make_img_label_overlay=1
                                    )
show_labels(df, image_label_overlay, show=1, saving_name=None, save_loc=None)"""

'df, image_label_overlay=scikitting(image_array,\n                                    black_region=True,\n                                    clearborder=1,\n                                    unite_boxes=0,\n                                    minDist=40,\n                                    area_threshold=200,\n                                    make_img_label_overlay=1\n                                    )\nshow_labels(df, image_label_overlay, show=1, saving_name=None, save_loc=None)'

In [10]:
# if unite_boxes
"""df, image_label_overlay=scikitting(image_array,
                                    black_region=True,
                                    clearborder=1,
                                    unite_boxes=1,
                                    minDist=40,
                                    area_threshold=200,
                                    make_img_label_overlay=1
                                    )
show_labels(df, image_label_overlay, show=1, saving_name=None, save_loc=None)"""

'df, image_label_overlay=scikitting(image_array,\n                                    black_region=True,\n                                    clearborder=1,\n                                    unite_boxes=1,\n                                    minDist=40,\n                                    area_threshold=200,\n                                    make_img_label_overlay=1\n                                    )\nshow_labels(df, image_label_overlay, show=1, saving_name=None, save_loc=None)'

# Where it may fail:

-  If image is tilted.---------> solution: Do tilt correction before everything else. Ask the user to be careful.
  

<img src="images/tilt_ptoblem_102__2016_STUHR_Gewerbesteuerbescheid.pdf.jpg">
- Handwriting, stamp, coffee stain etc. ---------> solution: Ask the user not write on the paper or write far from numbers.

<img src="images/5.jpg">


- The MinDist parameter may be too large.---------> solution: decreasing MinDist or make it dynamic.
  
<img src="images/1.jpg">

- More than one row may merge.---------> solution: Modifying the criteria of being in one row, e.g. more strict assignment.
  
<img src="images/126__2015_HAMBURG_Gewerbesteuerbescheid.pdf.jpg">

- Black background (rare)---------> solution: image processing. 
<img src="images/3.jpg">

- Detected blank masks (may not be problamatic in general)---------> solution: tesseract confidece output or image processing.
<img src="images/2.jpg">


